<a href="https://colab.research.google.com/github/Aipakazuma/meetup037/blob/master/99_%E7%B7%B4%E7%BF%92_tensorboard%E3%81%B8%E3%81%AE%E5%80%A4%E8%BF%BD%E5%8A%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorboardへ値追加

* tensorboardのクラスを継承して値や画像を表示させてみる

In [1]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation
from keras.utils import np_utils


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

Using TensorFlow backend.


In [2]:
!npm install -g localtunnel

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/client
+ localtunnel@1.9.1
added 54 packages from 32 contributors in 2.909s


In [0]:
!rm -rf ./logs/
!mkdir ./logs/

In [0]:
log_dir = './logs/'

In [0]:
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'.format(log_dir)
)

In [0]:
# 負荷かなにかでまれに落ちるので再実行してください
get_ipython().system_raw('lt --port 6006 > url.txt 2>&1 &')

In [29]:
!ps aux | grep tensorboard

root         149  0.8  2.4 1725044 329528 ?      Sl   07:42   0:14 /usr/bin/python2 /usr/local/bin/tensorboard --logdir ./logs/ --host 0.0.0.0 --port 6006
root        1702  0.0  0.0  39196  6572 ?        S    08:09   0:00 /bin/bash -c ps aux | grep tensorboard
root        1704  0.0  0.0  38568  5604 ?        S    08:09   0:00 grep tensorboard


In [30]:
!cat url.txt

your url is: https://wet-mole-0.localtunnel.me


## Tensorboardカスタマイズ


* 学習中の確率値の変化を確認したい
* 学習中、画像がちゃんと読み込めているか入力された画像を表示したい

In [0]:
import os
import tensorflow as tf
from keras.callbacks import TensorBoard


"""
on_epoch_begin: すべてのエポックの開始時に呼ばれます．
on_epoch_end: すべてのエポックの終了時に呼ばれます．
on_batch_begin: すべてのバッチの開始時に呼ばれます．
on_batch_end: すべてのバッチの終了時に呼ばれます．
on_train_begin: 訓練の開始時に呼ばれます．
on_train_end: 訓練の終了時に呼ばれます．

https://keras.io/ja/callbacks/#lambdacallback
"""


class CustomTensorBoard(TensorBoard):
    def __init__(self, **kwargs):
        super(CustomTensorBoard, self).__init__(**kwargs)

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        
        # logsはdictで、{key: value}を書き込むと
        # Tensorboardのscalarsに項目が追加されます
        # epoch毎にランダムで-100 ~ 100の間をランダムで取得して追加する.
        logs['int_test'] = np.random.randint(-100, 100)
        
        super(CustomTensorBoard, self).on_epoch_end(epoch=epoch, logs=logs)

## データダウンロード

In [9]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 38s 0us/step


In [0]:
# 画像を0-1の範囲で正規化
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# 正解ラベルをOne-Hot表現に変換
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [0]:
model = Sequential()
model.add(Flatten(input_shape=(32, 32, 3)))
model.add(Dense(32))
model.add(Activation('relu'))

model.add(Dense(32))
model.add(Activation('relu'))

model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                98336     
_________________________________________________________________
activation_1 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
activation_2 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                330       
_________________________________________________________________
activation_3 (Activation)    (None, 10)                0         
Total para

In [13]:
!ls -lht logs/*/

ls: cannot access 'logs/*/': No such file or directory


In [27]:
epochs = 300
batch_size = 128

try:
  
    # 実行するたびにディレクトリを作成するようにする.
    a = datetime.now()
    logs_name = a.strftime('%Y%m%d_%H%M%S')
    write_log_dir = '{}{}/'.format(log_dir, logs_name)
    tensorboard = CustomTensorBoard(log_dir=write_log_dir, batch_size=batch_size)
    
    print('書き込むディレクトリ名: {}'.format(write_log_dir))
    callbacks = [tensorboard]
    
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                                     callbacks=callbacks, validation_data=(X_test, y_test))
except KeyboardInterrupt:
    tensorboard.writer.close()

書き込むディレクトリ名: ./logs/20181214_075257/
Train on 50000 samples, validate on 10000 samples
Epoch 1/300
50000/50000 [==============================] - 3s 65us/step - loss: 1.4678 - acc: 0.4749 - val_loss: 1.5825 - val_acc: 0.4395
Epoch 2/300
50000/50000 [==============================] - 3s 65us/step - loss: 1.4679 - acc: 0.4717 - val_loss: 1.5549 - val_acc: 0.4436
Epoch 3/300
50000/50000 [==============================] - 3s 69us/step - loss: 1.4731 - acc: 0.4719 - val_loss: 1.5669 - val_acc: 0.4395
Epoch 4/300
50000/50000 [==============================] - 3s 67us/step - loss: 1.4673 - acc: 0.4734 - val_loss: 1.5547 - val_acc: 0.4439
Epoch 5/300
50000/50000 [==============================] - 3s 66us/step - loss: 1.4662 - acc: 0.4725 - val_loss: 1.5498 - val_acc: 0.4441
Epoch 6/300
50000/50000 [==============================] - 3s 69us/step - loss: 1.4688 - acc: 0.4723 - val_loss: 1.5617 - val_acc: 0.4443
Epoch 7/300
50000/50000 [==============================] - 3s 65us/step - loss: 1.465

In [0]:
a = model.predict(X_test)

In [16]:
a.mean(axis=0)

array([0.09469073, 0.11405851, 0.09156574, 0.10368641, 0.07537307,
       0.10907856, 0.08260404, 0.10579672, 0.11079076, 0.11235624],
      dtype=float32)